In [0]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [0]:
%mkdir ch9
!touch ch9/__init__.py

mkdir: cannot create directory ‘ch9’: File exists


In [0]:
import imp

## 9.1 对象表示形式

- `repr()`
  - 以便于开发者理解的方式返回对象的字符串表示形式
  - 需要对象实现 `__repr__` 特殊方法
- `str()` 
  - 以便于用户理解的方式返回对象的字符串表示形式
  - 需要对象实现 `__str__` 特殊方法


## 9-2 再谈向量类

###### 示例 9-2  vector2d_v0.py: 目前定义的都是特殊方法

In [0]:
%%writefile ch9/vector2d_v0.py
"""示例 9-2  vector2d_v0.py: 目前定义的都是特殊方法
"""

from array import array
import math


class Vector2d:
  typecode = 'd'

  def __init__(self, x, y):
    self.x = float(x)  # 使用 float 进行类型转换，尽量的捕获可能出现的错误
    self.y = float(y)

  def __iter__(self):
    return (i for i in (self.x, self.y))

  def __repr__(self):
    class_name = type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)  # 由于 Vector2d 是可迭代对象， 因此 *self 会将 x 和 y 分量提供给 format 函数

  def __str__(self):
    return str(tuple(self))  # 利用 Vector2d 是可迭代对象

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + 
        bytes(array(self.typecode, self)))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(self.x, self.y)

  def __bool__(self):
    return bool(abs(self))

Overwriting ch9/vector2d_v0.py


###### 示例 9-1　Vector2d 实例有多种表示形式

In [0]:
import ch9.vector2d_v0
imp.reload(ch9.vector2d_v0)
from ch9.vector2d_v0 import Vector2d

In [0]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [0]:
x, y = v1
x, y

(3.0, 4.0)

In [0]:
v1

Vector2d(3.0, 4.0)

In [0]:
v1_clone = eval(repr(v1))  # 使用 eval 函数，表明 repr 函数调用 Vector2d 实例得到的是对构造方法的准确表述
v1 == v1_clone

True

In [0]:
print(v1)

(3.0, 4.0)


In [0]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [0]:
abs(v1)

5.0

In [0]:
bool(v1), bool(Vector2d(0, 0))

(True, False)

## 9.3 备选构造方法

- `Vector2d` 实例能转换成字节序列；同理，也应该能从字节序列转换成 `Vector2d` 实例。

###### `Vector2d` 的 `frombytes` 方法

In [0]:
%%writefile -a ch9/vector2d_v0.py


  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)

Appending to ch9/vector2d_v0.py


In [0]:
import ch9.vector2d_v0
imp.reload(ch9.vector2d_v0)
from ch9.vector2d_v0 import Vector2d

In [0]:
v1 = Vector2d(3, 4)

In [0]:
v2 = Vector2d.frombytes(bytes(v1))

In [0]:
v1 == v2

True

## 9.4 `classmethod` 与 `staticmethod`

###### `classmethod`

- 定义操作类，而不是操作实例的方法
- 类方法的第一个参数是类本身，而不是实例
- 最常见的用途是定义备选构造方法
- 按约定，类方法的第一个参数名为 `cls`， 但可自定义为其它名称

###### `staticmethod`

- 会改变方法的调用方式，但是第一个参数不是特殊的值
- 静态方法只是普通的函数，只是在类的定义体中，而不是在模块层定义

###### 示例 9-4  比较 `classmethod` 与 `staticmethod` 的行为

In [0]:
class Demo:
  @classmethod
  def klassmeth(*args):
    return args
  
  @staticmethod
  def statmeth(*args):
    return args

In [0]:
Demo.klassmeth()

(__main__.Demo,)

In [0]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [0]:
Demo.statmeth()  # 行为与普通函数类似

()

In [0]:
Demo.statmeth('spam')

('spam',)

## 9.5 格式化显示

- 内置的 `format` 函数和 `str.format()` 方法把各个类型的格式化方式委托给相应的 `.__format__(format_spec)` 方法
  - `format_spec` 是格式化说明符，其是：
    - `format(my_obj, format_spec)` 第二个参数，或
    - `str.format()` 方法中的格式化字符串， `{}` 里代换字段中冒号后面的部分
  - 格式说明符使用的表示法叫做 [格式规范微语言](https://docs.python.org/3/library/string.html#formatspec)  

###### 示例

In [0]:
brl = 1/2.43
brl

0.4115226337448559

In [0]:
format(brl, '0.4f')

'0.4115'

In [0]:
'1 BRL = {rate:0.2f} USD'.format(rate=brl)

'1 BRL = 0.41 USD'

- 格式规范微语言为一些内置类型提供了专用的表示代码
  - `b` 和 `x` 分别表示二进制和十六进制的 `int` 类型，`f` 表示小数形式的 `float` 类
型，而 `%` 表示百分数形式

In [0]:
format(42, 'b')

'101010'

In [0]:
format(2/3, '.1%')

'66.7%'

- 格式规范微语言是可扩展的，因为各个类可以自行决定如何解释 `format_spec` 参数
  - 例如， `datetime` 模块中的类，它们的 `__format__` 方法使用的格式代码与 `strftime()` 函数一样

In [0]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%H:%S')

'08:08:07'

In [0]:
"It's now {:%I:%M %p}".format(now)

"It's now 08:00 AM"

- 类没有定义 `__format__` 方法，从 `object` 继承的方法会返回`str(my_object)`
- `Vector2d` 类定义了 `__str__` 方法，因此可以这样按如下方法调用

In [0]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

- 然而，如果传入格式说明符，`object.__format__` 方法会抛出 `TypeError`

In [0]:
format(v1, '.3f')

TypeError: ignored

###### 示例 9-6　Vector2d.__format__ 方法，可计算极坐标

In [0]:
%%writefile -a ch9/vector2d_v0.py


  def angle(self):
    return math.atan2(self.y, self.x)

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('p'):  # 如果格式代码以 'p' 结尾，使用极坐标
      fmt_spec = fmt_spec[:-1]
      coords = (abs(self), self.angle())  # 构造一个元组，表示极坐标
      outer_fmt = '<{}, {}>'
    else:
      coords = self
      outer_fmt = '({}, {})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(*components)

Appending to ch9/vector2d_v0.py


In [0]:
import ch9.vector2d_v0
imp.reload(ch9.vector2d_v0)
from ch9.vector2d_v0 import Vector2d

In [0]:
format(Vector2d(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [0]:
format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [0]:
format(Vector2d(1, 1), '.3e')

'(1.000e+00, 1.000e+00)'

In [0]:
format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

## 9.6 可散列的 `Vector2d`

- 按照定义， 目前 `Vector2d` 实例是不可散列的，因此不能放放集合中

In [0]:
import ch9.vector2d_v0
imp.reload(ch9.vector2d_v0)
from ch9.vector2d_v0 import Vector2d

In [0]:
v1 = Vector2d(3, 4)

In [0]:
hash(v1)

TypeError: ignored

In [0]:
set([v1])

TypeError: ignored

- 为了把 `Vector2d` 实例变成可散列的，必须使用 `__hash__` 方法（还需要 `__eq__` 方法）
- 此外，还需要让向量不可变

###### 示例 9-7 让 $x, y$ 不可变

In [0]:
%%writefile ch9/vector2d_v1.py
class Vector2d:
  typecode = 'd'

  def __init__(self, x, y):
    self.__x = float(x)  # 使用两个前导下划线，将属性标记为私有的
    self.__y = float(y)

  @property  # 将读值方法标记为特性
  def x(self):
    return self.__x

  @property
  def y(self):
    return self.__y

  def __iter__(self):
    return(i for i in (self.x, self.y))  # 读取x, y 分量的方法保持不变

  def __repr__(self):
    class_name = type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)  # 由于 Vector2d 是可迭代对象， 因此 *self 会将 x 和 y 分量提供给 format 函数

Overwriting ch9/vector2d_v1.py


###### 示例 9-8： 实现 __hash__ 方法

In [0]:
%%writefile -a ch9/vector2d_v1.py


  def __hash__(self):
    return hash(self.x) ^ hash(self.y)

Appending to ch9/vector2d_v1.py


###### 添加 `__hash__` 方法中之后，向量即变成可散列的

In [0]:
import ch9.vector2d_v1
imp.reload(ch9.vector2d_v1)
from ch9.vector2d_v1 import Vector2d

In [0]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(7, 384307168202284039)

In [0]:
set([v1, v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

###### 示例 9-9  vector2d_v3.py: 向量类的完整版

In [0]:
%%writefile ch9/vector2d_v3.py
"""
A two-dimensional vector class
"""

from array import array
import math


class Vector2d:
  typecode = 'd'

  def __init__(self, x, y):
    self.__x = float(x)  # 使用两个前导下划线，将属性标记为私有的
    self.__y = float(y)

  @property  # 将读值方法标记为特性
  def x(self):
    return self.__x

  @property
  def y(self):
    return self.__y

  def __iter__(self):
    return(i for i in (self.x, self.y))  # 读取x, y 分量的方法保持不变

  def __repr__(self):
    class_name = type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)  # 由于 Vector2d 是可迭代对象， 因此 *self 会将 x 和 y 分量提供给 format 函数
    
  def __str__(self):
    return str(tuple(self))  # 利用 Vector2d 是可迭代对象

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + 
        bytes(array(self.typecode, self)))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __hash__(self):
    return hash(self.x) ^ hash(self.y)

  def __abs__(self):
    return math.hypot(self.x, self.y)

  def __bool__(self):
    return bool(abs(self))

  def angle(self):
    return math.atan2(self.y, self.x)

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('p'):  # 如果格式代码以 'p' 结尾，使用极坐标
      fmt_spec = fmt_spec[:-1]
      coords = (abs(self), self.angle())  # 构造一个元组，表示极坐标
      outer_fmt = '<{}, {}>'
    else:
      coords = self
      outer_fmt = '({}, {})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(*components)
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)

Overwriting ch9/vector2d_v3.py


## 9.7 Python 的私有属性和 "受保护的" 属性

- 如果以 `__mood`(两个前导下划线) 命令实例属性， python 会把属性名存入实例的 `__dict__` 属性中，而且会在前面加上一个下划线和类名
- 名称改写是一种安全措施，不能确保万无一失： 其目的是为了避免意外访问，不能防止故意做错事
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200422172933.png width=600>

###### 示例 9-10　私有属性的名称会被“改写”，在前面加上下划线和类名

In [0]:
import ch9.vector2d_v3
imp.reload(ch9.vector2d_v3)
from ch9.vector2d_v3 import Vector2d

In [0]:
v1 = Vector2d(3, 4)

In [0]:
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [0]:
v1._Vector2d__x

3.0

###### 使用命名约定来避免意外覆盖属性

- Python 中约定使用一个下划线前缀编写 "受保护" 的属性(如 `self._x`)
- Pyhon 不会对使用单个下划线的属性名做特殊处理，不过是许多 Python 程序员严格遵守的规定

## 9.8 使用 `__slots__` 类属性节省空间

- python 在各个实例中名为 `__dict__` 的字典中存储实例属性
  - 但是为了提升访问速度，字典会消耗大量内存
- 如果处理数百万个属性不同的实例，通过 `__slots__` 类属性，能节省大量内存
  - 其方法是让解释器在元组中存储实例属性，而不用字典
- 继承自超类的 `__slots__` 属性没有效果， python 只会使用各个类中定义的 `__slots__` 属性，而不用字典

###### 示例 9-11 vector2d_v3_slots.py：只在 `Vector2d` 类中添加了 `__slots__` 属性

In [0]:
%%writefile ch9/vector2d_v3_slots.py
"""
A two-dimensional vector class
"""

from array import array
import math


class Vector2d:
  __slots__ = ('__x', '__y')  # 将 __slots__ 属性值设置为字符串构成的可迭代对象，即可告诉解释器此类中所有的实例属性，然后解释器会用类似于元组的数据结构进行储存

  typecode = 'd'

  def __init__(self, x, y):
    self.__x = float(x)  # 使用两个前导下划线，将属性标记为私有的
    self.__y = float(y)

  @property  # 将读值方法标记为特性
  def x(self):
    return self.__x

  @property
  def y(self):
    return self.__y

  def __iter__(self):
    return(i for i in (self.x, self.y))  # 读取x, y 分量的方法保持不变

  def __repr__(self):
    class_name = type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)  # 由于 Vector2d 是可迭代对象， 因此 *self 会将 x 和 y 分量提供给 format 函数
    
  def __str__(self):
    return str(tuple(self))  # 利用 Vector2d 是可迭代对象

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + 
        bytes(array(self.typecode, self)))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __hash__(self):
    return hash(self.x) ^ hash(self.y)

  def __abs__(self):
    return math.hypot(self.x, self.y)

  def __bool__(self):
    return bool(abs(self))

  def angle(self):
    return math.atan2(self.y, self.x)

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('p'):  # 如果格式代码以 'p' 结尾，使用极坐标
      fmt_spec = fmt_spec[:-1]
      coords = (abs(self), self.angle())  # 构造一个元组，表示极坐标
      outer_fmt = '<{}, {}>'
    else:
      coords = self
      outer_fmt = '({}, {})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(*components)
  
  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)


Overwriting ch9/vector2d_v3_slots.py


###### 示例 A-4  `memtest.py`: 创建大量 `Vector` 实例，报告内存用量

In [0]:
%%writefile ch9/mem_test.py
import importlib
import sys
import resource

NUM_VECTORS = 10 ** 7

if len(sys.argv) == 2:
  module_name = sys.argv[1].replace('.py', '')
  module = importlib.import_module(module_name)
else:
  print('Usage: {} <vector-module-to-test>'.format())
  sys.exit(1)

fmt = 'Selected Vector2d type: {.__name__}{.__name__}'
print(fmt.format(module, module.Vector2d))

mem_init = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
print('Creating {:,} Vector2d instances'.format(NUM_VECTORS))

vectors = [module.Vector2d(3.0, 4.0) for i in range(NUM_VECTORS)]

mem_final = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
print('Initial RAM usage: {:14,}'.format(mem_init))
print(' Final RAM usage: {:14,}'.format(mem_final))


Overwriting ch9/mem_test.py


###### 示例 9-12　mem_test.py 使用指定模块（如 vector2d_v3.py）中定义的 Vector2d 类创建 10,000,000 个实例

In [0]:
!time python3 -m ch9.mem_test ch9.vector2d_v3

Selected Vector2d type: ch9.vector2d_v3Vector2d
Creating 10,000,000 Vector2d instances
Initial RAM usage:         13,544
 Final RAM usage:      1,881,848

real	0m12.020s
user	0m10.605s
sys	0m1.388s


In [0]:
!time python3 -m ch9.mem_test ch9.vector2d_v3_slots

Selected Vector2d type: ch9.vector2d_v3_slotsVector2d
Creating 10,000,000 Vector2d instances
Initial RAM usage:         13,452
 Final RAM usage:        714,352

real	0m9.870s
user	0m9.518s
sys	0m0.328s


- 在 `Vecotr2d` 中定义 `__slots__` 属性后， RAM 用量降低，运行速度也更快

##### `__slots__` 的问题

- 如果使用得当， `__slots__` 能显著节省内存，但需注意以下几点
  1. 每个子类都需定义 `__slots__` 属性，因为解释器会忽略继承的 `__slots__` 属性
  2. 实例只能拥有 `__slots__` 中列出的属性
    - 除非把 `__dict__` 加入 `__slots__` 中，但这样做会失去节省内存在功效
  3. 如果不把 `__weakref__` 加入 `__slots__`， 实例就不能作为弱引用的目标

## 9.9 覆盖类属性

- 如果实例属性不存在，则会去获取类属性
- 如果为不存在的实例属性赋值，则会新建实例属性，如果实例属性的值和类属性相同，则会把同名类属性遮盖
- 借助这一特性，可以给 `Vector2d` 中的 `typecode` 属性定制不同的值  

###### 示例 9-13　设定从类中继承的 `typecode` 属性，自定义一个实例属性

In [0]:
import ch9.vector2d_v3
imp.reload(ch9.vector2d_v3)
from ch9.vector2d_v3 import Vector2d

In [0]:
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [0]:
len(dumpd)

17

In [0]:
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [0]:
len(dumpf)

9

In [0]:
Vector2d.typecode  # 类属性的值并没有发生改变

'd'

##### 更符合 Python 风格的修改方式

- 可以创建一个子类，只用于定制类的数据属性，如此可继随父类的类属性
- 这样修改效果持久，而且更有针对性

###### 示例 9-14　`ShortVector2d` 是 `Vector2d` 的子类，只用于覆盖 `typecode` 的默认值

In [0]:
import ch9.vector2d_v3
imp.reload(ch9.vector2d_v3)

<module 'ch9.vector2d_v3' from '/content/drive/My Drive/Colab Notebooks/fluent_python_notes/ch9/vector2d_v3.py'>

In [0]:
from ch9.vector2d_v3 import Vector2d

class ShortVector2d(Vector2d):
  typecode = 'f'

In [0]:
sv = ShortVector2d(1/11, 1/27)  # Vector2d 中没有硬编码 class_name 的值，而是采用 type(self). 如此，便可直接从父类继承
sv

ShortVector2d(0.09090909090909091, 0.037037037037037035)

In [0]:
len(bytes(sv))

9